In [1]:
import numpy as np
import pandas as pd
import orangecontrib.associate.fpgrowth as oaf
import datetime
import argparse
#parser = argparse.ArgumentParser()
#parser.add_argument('--train_dir', type=str, default='./trainDataHJ.txt')
#parser.add_argument('--rules_dir', type=str, default='./rulesDataHJ.txt')
#parser.add_argument('--support', type=float, default=0.2)
#parser.add_argument('--confidence',type=float,default=0.2)
#args = parser.parse_args()

In [11]:
y=np.random.randint(1,6,[1000,])
#area=np.random.uniform(1,10,[1000,])
#length=np.random.uniform(1,10,[1000,])
#metal_thi=np.random.uniform(0,2,[1000,])
#metal_cat=np.random.randint(1,4,[1000,])
mach_cat=np.random.randint(1,4,[1000,])
meth_cat=np.random.randint(1,4,[1000,])


In [5]:
df=pd.read_csv('trainDataHJ.txt',names=['area','length','speed','cross_area','shc','loc','fuck','shit','t'],sep=' ')

In [7]:
del df['fuck']
del df['shit']
del df['t']

In [12]:
df['mach_cat']=mach_cat
df['meth_cat']=meth_cat
df['quality']=y

In [13]:
df

,area,length,speed,cross_area,shc,loc,mach_cat,meth_cat,quality
0,5.378033,7.387552,11.018732,2.618714,720.033047,1.0,3,2,5
1,5.456136,1.180244,11.081069,3.670108,327.343671,1.0,3,2,5
2,3.818622,2.127591,18.426917,4.660761,694.724254,3.0,1,3,3
3,6.079148,6.570358,11.851046,2.660607,558.094718,1.0,3,3,1
4,5.956198,6.050244,12.657710,3.588428,530.032262,3.0,3,2,2
...,...,...,...,...,...,...,...,...,...
995,5.943159,5.406279,12.669517,2.906841,651.679734,1.0,3,3,2
996,6.166234,9.371067,10.678986,2.115976,472.596541,2.0,1,2,5
997,3.319771,6.799239,11.479971,2.462304,587.056807,3.0,3,3,3
998,2.818361,9.438325,7.075656,2.928550,478.679443,1.0,2,1,3


In [14]:
#df.to_csv('trainDataHF.txt',index=False)

In [18]:
df['speed'].describe()

count    1000.000000
mean       12.102103
std         3.113857
min         2.500001
25%        10.075725
50%        12.139864
75%        14.320187
max        21.719486
Name: speed, dtype: float64

In [19]:
df['cross_area'].describe()

count    1000.000000
mean        2.953745
std         1.158299
min         1.004401
25%         1.931056
50%         2.907164
75%         3.905285
max         4.997788
Name: cross_area, dtype: float64

In [21]:
df['shc'].describe()

count    1000.000000
mean      495.408688
std        97.882070
min       193.838989
25%       428.180880
50%       495.591120
75%       563.557193
max       785.880462
Name: shc, dtype: float64

In [22]:
def data_pre(path):
    df=pd.read_csv(path)
    #iscretization
    bins=[0,2,4,6,8,10]
    df['area']=pd.cut(df['area'],bins,right=False,labels=['1','2','3','4','5'])
    bins=[0,2,4,6,8,10]
    df['length']=pd.cut(df['length'],bins,right=False,labels=['1','2','3','4','5'])
    bins=[0,10,12,14,22]
    df['speed']=pd.cut(df['speed'],bins,right=False,labels=['1','2','3','4'])
    bins=[0,2,3,4,5]
    df['cross_area']=pd.cut(df['cross_area'],bins,right=False,labels=['1','2','3','4'])
    bins=[0,450,500,600,800]
    df['shc']=pd.cut(df['shc'],bins,right=False,labels=['1','2','3','4'])
    #type trans
    df['loc']=df['loc'].apply(int).apply(str)
    df['mach_cat']=df['mach_cat'].apply(str)
    df['meth_cat']=df['meth_cat'].apply(str)
    df['quality']=df['quality'].apply(str)
    #one_hot
    df=pd.get_dummies(df)
    strDecode=df.columns.values.tolist()
    data=df.values
    data.dtype=bool
    return data,strDecode

In [23]:
def model(data,support=0.05,confidence=0.2):
    fre_ite=dict(oaf.frequent_itemsets(data,support))  #这里设置置信度
    rules = oaf.association_rules(fre_ite,confidence)
    result=list(rules)
    return result

In [24]:
def dealResult(rules,strDecode):
    returnRules = []
    for i in rules:
        j= list(i[1])
        if len(j)==1 and strDecode[j[0]].startswith('quality') :
            temStr = [];
            temp=''
            for k in i[0]:
                temp = temp+strDecode[k]+'&'
            temp = temp[:-1]
            temp = temp + ' ==> '
            temp = temp +strDecode[j[0]] 
            temStr.append(temp)
            temStr.append(i[2])
            temStr.append(i[3])
            returnRules.append(temStr)
    returnRules=list(sorted(returnRules,key=lambda x:(-x[2],-x[1])))
    return returnRules
    


In [25]:
support=0.05
confidence=0.2
train_dir='trainDataHF.txt'
rules_dir='rulesDataHF.txt'
data,strDecode=data_pre(train_dir)
result=model(data,support,confidence)
printRules= dealResult(result,strDecode)
file=open(rules_dir,'w')
print('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
file.write('Rules'+'\t'+'support'+'\t'+'confidence'+'\n')
for line in printRules:
    s='\t'.join([str(i) for i in line])
    print(s+'\n')
    file.write(s+'\n')
file.close()
    

Rules	support	confidence

cross_area_4 ==> quality_3	61	0.2687224669603524

loc_3 ==> quality_2	63	0.2647058823529412

area_3&meth_cat_1 ==> quality_3	55	0.248868778280543

loc_2 ==> quality_3	56	0.24778761061946902

area_3&meth_cat_2 ==> quality_1	53	0.24766355140186916

meth_cat_1 ==> quality_3	78	0.24148606811145512

shc_1&area_3 ==> quality_3	56	0.23931623931623933

mach_cat_1&area_3 ==> quality_3	52	0.23853211009174313

mach_cat_1&area_3 ==> quality_2	52	0.23853211009174313

cross_area_1 ==> quality_4	63	0.2342007434944238

area_3&meth_cat_3 ==> quality_3	54	0.23376623376623376

mach_cat_1 ==> quality_2	76	0.2331288343558282

meth_cat_2 ==> quality_2	74	0.23125

loc_4 ==> quality_1	58	0.23107569721115537

shc_1 ==> quality_3	78	0.23076923076923078

mach_cat_1 ==> quality_3	75	0.23006134969325154

speed_2 ==> quality_3	54	0.2222222222222222

speed_2 ==> quality_1	54	0.2222222222222222

area_3&mach_cat_2 ==> quality_5	50	0.22123893805309736

cross_area_3 ==> quality_1	54	0.220408163